In [ ]:
# President: 2016 (Trump), 2020 (Biden), 2024 (Trump)
# Governor: 2018 (Whitmer), 2022 (Whitmer)
# Secretary of State: 2018 (Benson), 2022 (Benson)
# Attorney General: 2018 (Nessel), 2022 (Nessel)
# U.S. Senate: 2014 (Peters), 2018 (Stabenow), 2020 (Peters), 2024 (Slotkin)
# U.S. House: every cycle
# State Senate: 2014, 2018, 2022
# State House: every cycle

# OFFICES = ['U.S. House', 'State House']
# YEARS = ['2018', '2020', '2022', '2024']

# OFFICES = ['U.S. Senate']
# YEARS = ['2014', '2018', '2020', '2024']

# OFFICES = ['State Senate']
# YEARS = ['2014', '2018', '2022']

# OFFICES = ['President']
# YEARS = ['2016', '2020', '2024']

# Not enough data
# # OFFICES = ['Governor', 'Secretary of State', 'Attorney General']
# # YEARS = ['2018', '2022']

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
def makeColors():
    n_colors = 17
    half = n_colors // 2
    
    # First half red
    reds = np.linspace(255, 129, half, dtype=int)
    greens = np.linspace(0, 129, half, dtype=int)
    blues = np.linspace(0, 129, half, dtype=int)
    first_half = [(r, g, b) for r, g, b in zip(reds, greens, blues)]
    
    # Middle is gray
    middle = [(128, 128, 128)]
    
    # Second half blue
    reds = np.linspace(127, 0, half, dtype=int)
    greens = np.linspace(127, 0, half, dtype=int)
    blues = np.linspace(127, 255, half, dtype=int)
    second_half = [(r, g, b) for r, g, b in zip(reds, greens, blues)]
    
    # Make gradient
    full_gradient_rgb = first_half + middle + second_half
    colors = ['#{:02x}{:02x}{:02x}'.format(r, g, b) for r, g, b in full_gradient_rgb]

    return colors

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import to_rgba

def plotPrecinctBounds(df, year, office):
    colors = makeColors()
    
    color_map = {
        "gargantuanly more republican": colors[0],
        "massively more republican": colors[1],
        "much much more republican": colors[2],
        "much more republican": colors[3],
        "more republican": colors[4],
        "slightly more republican": colors[5],
        "very slightly more republican": colors[6],
        "infinitesimally more republican": colors[7],
        "no change": colors[8],
        "infinitesimally more democrat": colors[9],
        "very slightly more democrat": colors[10],
        "slightly more democrat": colors[11],
        "more democrat": colors[12],
        "much more democrat": colors[13],
        "much much democrat": colors[14],
        "massively more democrat": colors[15],
        "gargantuanly more democrat": colors[16],
        "unknown": '#000000',
    }

    df['color'] = df['predicted_partisanship_change'].apply(lambda x: color_map[x])

    fig, ax = plt.subplots(figsize=(80, 80))
    divider = make_axes_locatable(ax)

    df.boundary.plot(ax=ax, color="black", linewidth=0.1)
    df.plot(ax=ax, color=df['color'], edgecolor="black", linewidth=0.01)

    ax.margins(0)
    ax.set_title('Precinct Preds', fontsize=64)
    ax.set_axis_off()

    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.savefig('output/maps/precincts/' + str(year) + "_" + office.replace(' ', '_').replace('.', '') + "_Prediction_Map.png")
    
    plt.close(fig)

In [ ]:
for year in YEARS:
    for office in OFFICES:
        df_precinct_outcomes = gpd.read_file(f'data/generated_data/df_02_vote_changes_calc_{year}_{office.replace(' ', '_').replace('.', '')}.geojson', driver='GeoJSON')
        df_precinct_outcomes['standardized_id_num'] = df_precinct_outcomes['standardized_id_num'].astype(str).str.zfill(13)
        
        df_precinct_pred_change = pd.read_csv(f'data/generated_data/predicted_partisanship_change_{year}_{office.replace(' ', '_').replace('.', '')}.csv')
        df_precinct_pred_change['standardized_id_num'] = df_precinct_pred_change['standardized_id_num'].astype(int).astype(str).str.zfill(13)

        df_precinct_pred = df_precinct_outcomes.merge(df_precinct_pred_change, on="standardized_id_num", how="inner")
        
        plotPrecinctBounds(df_precinct_pred, year, office)

print('DONE')